In [ ]:
flooding_change_detection = ee.Image('projects/ee-tut-452819/assets/flood_extent_houston')
hand_inundation = ee.Image('projects/ee-tut-452819/assets/HAND_no_threshold')
hand_inundation = hand_inundation.gte(best_threshold)
hand_inundation = hand_inundation.clip(aoi)
flooded_change_detection = flooding_change_detection.clip(aoi)

In [ ]:
after_start = '2017-08-28'
after_end = '2017-09-05'

# RF model
flood_truth = flooding_change_detection.unmask()
precip = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate(after_start, after_end).sum().clip(aoi)  # total precipitation over flood month

# Land Cover (static, 2020)
landcover = ee.Image('ESA/WorldCover/v100/2020').clip(aoi)

In [ ]:
target_proj = flood_truth.projection().atScale(100)

precip_band = precip.resample('bicubic').reproject(target_proj)
precip_band = precip_band.updateMask(precip_band.gt(-1)).clip(table)

# Reproject and resample land cover
landcover_band = landcover.reproject(crs=target_proj)